# The TITANIC Beginner Exercies

In [1]:
import pandas as pd
import numpy as np
import string
import matplotlib

df = pd.read_csv("/Users/bnorman/Desktop/IBM/titanic/Datas/train.csv")
df_test = pd.read_csv("/Users/bnorman/Desktop/IBM/titanic/Datas/test.csv")
df_result = pd.read_csv("/Users/bnorman/Desktop/IBM/titanic/Datas/gender_submission.csv")


df_test.set_index('PassengerId')
df_result.set_index('PassengerId')
df.set_index('PassengerId')



,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


## Feature Engineering part

### Creating new features

In this part we have got the clean data set. Now we are going to add some more features

#### The Title feature

We cant do anything to the name feature, because its a string and we can't make the algorithm to learn it. But from the names we can get the title of each person. (Prefixes)

In [2]:
title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                    'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer']
newtitle_list=['Mr','Mrs', 'Miss', 'Master']

def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if str.find(big_string, substring) != -1:
            return substring
    return np.non

df['Title']=df['Name'].map(lambda x: substrings_in_string(x, title_list))
df_test['Title']=df_test['Name'].map(lambda x: substrings_in_string(x, title_list))


def replace_titles(x):
    title=x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
        return 'Mr'
    elif title in ['Countess', 'Mme']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title
    
df['Title']=df.apply(replace_titles, axis=1)
df_test['Title']=df_test.apply(replace_titles, axis=1)

#### The Deck feature

We cant get much from the Cabin number. It's a complicated identifier, generated for the ticket, but it contains the deck.

For that we need to specify the "missing data". After some research i found, that it is not missing. Simply those person, that did not have anything about that had no Cabin. 

In [3]:
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
df["Cabin"].fillna("Unknown",inplace = True)
df_test["Cabin"].fillna("Unknown",inplace = True)
df['Deck']=df['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
df_test['Deck']=df_test['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))

#### The SizeOfFamily feature

In the raw data we have got informations about the numbers of the siblings and the numbers of the parents. We can sum that and we get the size of the family above

In [4]:
df['Family_Size'] = df['SibSp']+df['Parch']
df_test['Family_Size'] = df_test['SibSp']+df_test['Parch']

Now, we have got all the new features initialized. 

In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Deck,Family_Size
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S,Mr,Unknown,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S,Miss,Unknown,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,C,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S,Mr,Unknown,0


We got information from the 'Name' and the 'Ticket'. Its time to drop them.

In [6]:
df = df.drop(['Name'], axis = 1)
df = df.drop(['Ticket'],axis = 1)
df = df.drop(['Cabin'], axis = 1)
df_test = df_test.drop(['Name'], axis = 1)
df_test = df_test.drop(['Ticket'],axis = 1)
df_test = df_test.drop(['Cabin'], axis = 1)

### Handling missing values

In [7]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
Title            0
Deck             0
Family_Size      0
dtype: int64

As the table above shows me, there are some missing informations about the ages. We cant drop 177 records from the database with size 889.

Filling the age was a really hard exercise. First i tought the avg age will be a good value for that, but it wouldn't be the best option.


So i grabbed the titles and filled the missing value based on the person's title. It's good, because form a title (for example dr) we know, that this person has to be older than x ages.

In [17]:
def FillAge(list):
    ageMean = df.Age.mean()
    ageMeans = df_test.Age.mean()
    #for title in list:
        #ageMean = df.loc[df['Title']==title].Age.mean()
        #df[(pd.isnull(df.Age)) & df['Title'] == title]['Age'] = ageMean #ASK BERNAT!!!!!!!!!!!!
        
        #ageMeans = df_test.loc[df_test['Title']==title].Age.mean()
        #df_test[(pd.isnull(df_test.Age)) & df['Title'] == title]['Age'] = ageMeans #ASK BERNAT!!!!!!!!!!!!
    df.loc[pd.isnull(df.Age),'Age'] = ageMean
    df_test.loc[pd.isnull(df_test.Age),'Age'] = ageMeans

    
FillAge(newtitle_list)

df = df.dropna()
df.isnull().sum()

df_test.isnull().sum()

PassengerId     0
Age             0
SibSp           0
Parch           0
Fare            0
Family_Size     0
Sex_male        0
Embarked_Q      0
Embarked_S      0
Pclass_2        0
Pclass_3        0
Deck_B          0
Deck_C          0
Deck_D          0
Deck_E          0
Deck_F          0
Deck_G          0
Deck_T          0
Deck_Unknown    0
Title_Miss      0
Title_Mr        0
Title_Mrs       0
dtype: int64

Oh-Oh there is a missing Fare in the df_test. Lets remove it.

In [9]:
missindId = df_test[df_test.Fare.isnull()].index.values.astype(int)
df_test = df_test.drop(missindId)
df_result = df_result.drop(missindId)

Finally we don't have any missing value

In [18]:
df_test.isnull().sum()

PassengerId     0
Age             0
SibSp           0
Parch           0
Fare            0
Family_Size     0
Sex_male        0
Embarked_Q      0
Embarked_S      0
Pclass_2        0
Pclass_3        0
Deck_B          0
Deck_C          0
Deck_D          0
Deck_E          0
Deck_F          0
Deck_G          0
Deck_T          0
Deck_Unknown    0
Title_Miss      0
Title_Mr        0
Title_Mrs       0
dtype: int64

In [19]:
df.isnull().sum()

PassengerId     0
Age             0
SibSp           0
Parch           0
Fare            0
Family_Size     0
Sex_male        0
Embarked_Q      0
Embarked_S      0
Pclass_2        0
Pclass_3        0
Deck_B          0
Deck_C          0
Deck_D          0
Deck_E          0
Deck_F          0
Deck_G          0
Deck_T          0
Deck_Unknown    0
Title_Miss      0
Title_Mr        0
Title_Mrs       0
dtype: int64

### Creating the dummies (One hot vectors)

Now everything is ready to teach our algorithm, but there are still strings in our data fragment. Lets create the one hot vectors from them. It's necessary to use this technique, beacuse initializing them just like scalars would mean, that the distance between them is not equal. I'd lead to a bad conclusion.

In [12]:
df = pd.get_dummies(df,columns= ['Sex','Embarked', 'Pclass','Deck', 'Title'],drop_first=True)
Y = df['Survived'].copy()
df = df.drop(['Survived'], axis = 1)
df.head()

df_test = pd.get_dummies(df_test,columns= ['Sex','Embarked', 'Pclass','Deck', 'Title'],drop_first=True)



OUR DATA IS READY 

## Logistic regression

For the implementing of the logistic regression i'll use the scikit learn

In [22]:
from sklearn.linear_model import LogisticRegression
df_test.insert(17,"Deck_T",0)


ValueError: cannot insert Deck_T, already exists

In [34]:

logreg = LogisticRegression(random_state=0)
logreg.fit(df,Y)
#print(Y)
#print(df_test)
logreg.score(df_test,df_result["Survived"])
#Y_pred = logreg.predict(df_test)
#arrayY = df_result["Survived"].tolist()
#print(df_test)
#print(arrayY)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9280575539568345

In [24]:
predicted ="0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 0 1 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 1 0 0 0"
predicted = predicted.replace(" ","")

test= "0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0"
test = test.replace(", ","")

numAll = 0
numError = 0
for i in range(len(predicted)):
    numAll+=1
    if test[i] != predicted[i]:
        numError +=1
print('Every:' + str(numAll))
print('Errors:' + str(numError))
print('Accuracy:' + str(((numAll-numError)/numAll)*100)+'%')

Every:417
Errors:30
Accuracy:92.80575539568345%


In [ ]:
print(arrayY)

In [ ]:
predicted ="0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 1 0 0 0"
predicted = predicted.replace(" ","")

test = "0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0"
test = test.replace(", ","")

numAll = 0
numError = 0
for i in range(len(predicted)):
    numAll+=1
    if test[i] != predicted[i]:
        numError +=1
print('Every:' + str(numAll))
print('Errors:' + str(numError))
print('Accuracy:' + str(((numAll-numError)/numAll)*100)+'%')